### Prototype code to scrape the top headlines and their corresponding article text from News API

In [1]:
import os.path
from os import path
import pandas as pd
import json
import boto3 
import os
from io import StringIO


In [2]:
import pprint
import requests

In [3]:
# API Key for News API
secret = 'd05c3ae08abd461f90b3dc555af1bb59'

In [4]:
# s3 secret
ACCESS_KEY = "<KEY>" 
SECRET_KEY = "<SECRET>" 
BUCKET_NAME ='<bucket>'


In [5]:
s3 = boto3.resource(
    's3',
    region_name='us-east-1',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY
).Bucket(BUCKET_NAME)

json.load_s3 = lambda f: json.load(s3.Object(key=f).get()["Body"])
json.dump_s3 = lambda obj, f: s3.Object(key=f).put(Body=json.dumps(obj))

In [6]:
# Define the endpoint to extract all the top headlines 
url = 'https://newsapi.org/v2/top-headlines?'

In [7]:
# Specify the query and number of returns - Limit the headlines to country US, for now
parameters = {
    'language': 'en',
    'country':'us',
    'pageSize': 100,
    'apiKey': secret 
}

In [8]:
categories = ['business','entertainment','general','health','science','sports','technology']


In [9]:
from newspaper import Article
from newspaper import Config


article_title = []
article_authors = []
article_text = []
article_summary = []
article_date = []
article_top_image = []
failed_url = []
category_articles ={}
pages= range(1,2)

In [10]:
#category_dataframe = pd.read_json(f'resources/business_top_headline_data.json').T
#category_dataframe.head()

In [11]:
count =0
for category in categories:
    name = category
    print(name)
    fileName= f'resources/{category}_top_headline_data_new.json'
    json_buffer=json.load_s3('Project3/'+fileName)
    category_dataframe = pd.read_json(json_buffer).T    
    parameters['category'] = category
    response = requests.get(url, params=parameters)
    if response.status_code != requests.codes.ok:
        print(f"Bad result : {response.url}")
        continue
    response_json = response.json()

    df = pd.DataFrame.from_dict(response_json)
    df = pd.concat([df.drop(['articles'], axis=1), df['articles'].apply(pd.Series)], axis=1) 

    rows = []
    for index, row in df.iterrows():
        h = row['url']
        user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
        config = Config()
        config.verbose=True
        config.browser_user_agent = user_agent
        newsarticle = Article(h, config=config)
        try :
            newsarticle.download()
            newsarticle.parse()
            if not newsarticle.text :
                print(f"Unable to parse article: {h}")
                continue 
            artdict = {}                
            artdict['articleText']=newsarticle.text
            artdict['articleSummary']=''   # Place Holder for bert summary
            artdict['articleSentiment']='' # Place Holder for calculated Sentiment Analysis
            for column in df:
                #print(column)
                if column in ['status', 'totalResults','content'] :
                    continue 
                artdict[column]= row[column]
            rows.append(artdict)
            count += 1
            if (count % 50 == 0 ):
                print('Number of articles: '+str(count))

        except :
             print(f'***FAILED TO DOWNLOAD***{newsarticle.url}')
             pass

    print(f"Using DataFrame {category_dataframe.shape} Adding {len(rows)}" )
    category_dataframe = category_dataframe.append(pd.DataFrame(rows),ignore_index=True).drop_duplicates(subset='url')
#         df.to_json(f'resources/{category}_top_headline_data_new.json', orient='index')
#         category_articles[name] = category_dataframe
    print(category_dataframe.shape)
    json_buffer = StringIO()
    category_dataframe.to_json(json_buffer, orient='index')
    json.dump_s3(json_buffer.getvalue(),'Project3/'+fileName)



print('Total number of articles: '+str(count))
    

business
***FAILED TO DOWNLOAD***https://www.washingtonpost.com/business/2020/07/11/after-fastest-recession-us-history-economic-recovery-may-be-fizzling/
Unable to parse article: http://turnto10.com/news/local/providence-post-office-evacuated-2-employees-sent-to-hospital
Unable to parse article: http://news3lv.com/news/local/eldorado-must-sell-3-indiana-casinos-as-part-of-caesars-deal-07-11-2020
***FAILED TO DOWNLOAD***https://www.wptv.com/news/region-c-palm-beach-county/west-palm-beach/template-west-palm-beach
Unable to parse article: https://www.profgalloway.com/lets-get-ready-to-rundle/
Number of articles: 50
***FAILED TO DOWNLOAD***https://news.google.com/__i/rss/rd/articles/CBMiXmh0dHBzOi8vd3d3LmFnd2ViLmNvbS9hcnRpY2xlL2Nvcm4tbWFya2V0LXVucGhhc2VkLXByb2R1Y3Rpb24tZHJvcC1sYXJnZS1jaGluZXNlLWJ1eS1oZXJlcy13aHnSAQA?oc=5
Using DataFrame (234, 10) Adding 64
(249, 10)
entertainment
Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdG

Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9NGtWaXMyRHdjM1HSAQA?oc=5
Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9T0ZMcXBtZnFuUm_SAQA?oc=5
Number of articles: 300
Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9UmdrTzNRTTluOGfSAQA?oc=5
Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9a29OV2tpdU00enPSAQA?oc=5
Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9WS1oSUJPRFQxSUHSAQA?oc=5
Using DataFrame (243, 10) Adding 62
(270, 10)
technology
Unable to parse article: https://news.google.com/__i/rss/rd/articles/CBMiK2h0dHBzOi8vd3d3LnlvdXR1YmUuY29tL3dhdGNoP3Y9U1VzSVlwZVIwckHSAQA?oc=5
Unable to parse article: https://news.google.com/__i/rss/rd/article